 Hate speech classification using  deep learning models

Resource 
https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17


 https://www.youtube.com/watch?v=2qnXmB65Ino&t=41s
 
 # Import  deep learning libraries

In [ ]:
import os
import re
import string
import emoji
import pandas as pd
import numpy as np
import pydot
from collections import Counter
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import mlxtend
from mlxtend.plotting import plot_confusion_matrix
from tensorflow import keras 
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import regularizers
from tensorflow.keras import preprocessing
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn import preprocessing

from tensorflow.keras.preprocessing.sequence import pad_sequences
label_encoder=preprocessing.LabelEncoder()
from sklearn.metrics import confusion_matrix
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings


# Read data

In [14]:
import pandas as pd
pd.set_option('display.max_colwidth', None)#To display full length of the text
warnings.filterwarnings("ignore")
df = pd.read_csv('final_dataset.csv', sep=',', encoding='utf-8')

In [15]:
df.tail()

,Unnamed: 0,tweet_id,anno1,anno2,curator,tweet
3029,567,1325473860804882433,hate,hate,hate,ግዜ ለኩሉ በተራችሁ ወጣችሁ ጩሁ ማፈሪያዎች
3030,568,1345406853719605248,normal,normal,normal,ይቅርታ ከጎረቤቱ ውሻ የበለጠ ቆንጆ ነኝ
3031,569,1388926201213956102,normal,normal,normal,ያልፈሳንበት ዳገት የለም አለች አህያ
3032,570,1358433535296761857,normal,normal,normal,ውሻ በበላበት ቢጮህ ምን ይደንቃል ።
3033,571,1394711686007902210,hate,hate,normal,ደደብ ነን እንጂ ድሀ አይደለንም!


# Remove emoji and symbols

In [16]:
def demoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U00010000-\U0010ffff"
                             "]+", flags=re.UNICODE)
    return(emoji_pattern.sub(r'', text))
 
df[u'tweet'] = df[u'tweet'].astype(str)
df[u'tweet'] = df[u'tweet'].apply(lambda x:demoji(x))

# Split the data into train and test set

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=10, shuffle=True)

In [ ]:
train.to_csv('train.csv',index=False)
test.to_csv('test.csv',index=False)

In [ ]:
df_train = pd.read_csv('train.csv',sep=',', encoding='utf-8')#header=0,
df_test = pd.read_csv('test.csv',sep=',', encoding='utf-8')#header=0,
#df_train.info()
df_train.head(5)

In [ ]:
df_train.columns =['label', 'tweet']
df_test.columns =['label', 'tweet']

In [ ]:
train, test = (df_train, df_test)

In [ ]:
#train['tweet'].value_counts()
len(df_train)

In [ ]:
train['label'].value_counts()

# Pad squence and vectorization

In [ ]:
from keras.preprocessing.text import Tokenizer
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df_train['tweet'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
from keras.preprocessing.sequence import pad_sequences
X = tokenizer.texts_to_sequences(df_train['tweet'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH) 
max_len = 0
for t in X:
    if max_len < len(t):
        max_len = len(t)
X1 = tokenizer.texts_to_sequences(df_test['tweet'].values)
X1 = pad_sequences(X1, maxlen=max_len)
print('Shape of data tensor:', X.shape)

In [ ]:
Y = pd.get_dummies(df_train['label']).values
Y1 = pd.get_dummies(df_test['label']).values
print('Shape of label tensor:', Y.shape)
Y2 = pd.get_dummies(test['label']) 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 8)
X_train = X
X_test=X1
Y_train=Y
Y_test=Y1
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

# LSTM

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
epochs = 10
batch_size = 64

print(model.summary())
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)#callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)]

## LSTM  result

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
valid_predictl1= model.predict(X1)
print(valid_predictl1[:5])

In [ ]:
#Generate predictions for all samples
predictions = model.predict(X1)
print(predictions)
predict_resultsl1=predictions.argmax(axis=1)
print(predict_resultsl1[11])

In [ ]:
from sklearn.metrics import classification_report
import numpy as np
test['predict']= predict_resultsl1
test['predict'] = np.where((test.predict == 0),'hate',test.predict)
test['predict'] = np.where((test.predict =='1'),'normal',test.predict)
test['predict'] = np.where((test.predict =='2' ),'offensive',test.predict)
test['predict'] = np.where((test.predict =='3'),'unsure',test.predict)
labels = ['hate','normal','offensive','unsure']
print(classification_report(test['label'].tolist(),test['predict'].tolist(),labels=labels))

In [ ]:
from tensorflow.keras.models import load_model
    
model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
    
model = load_model('my_model.h5')

#print(valid_predictl1[:1])

## lstm confusion matrix

In [ ]:
pred = model.predict(X1)
pred= np.argmax(pred, axis = 1)
test_labels = np.argmax(Y1,axis = 1)
cm=confusion_matrix(test_labels,pred)
plot_confusion_matrix(cm,figsize=(8,8), class_names=labels)

## lstm loss and accuracy graph

In [ ]:
import matplotlib.pyplot as plt
plt.title('Loss')
plt.xlabel('epoch')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Accuracy')
plt.xlabel('epoch')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();

# BiLstm 


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.layers import LSTM,Bidirectional,Dense,Embedding,Dropout

model_b=Sequential()
model_b.add(Embedding(70000,32,input_length=100))
model_b.add(Dropout(0.7))
model_b.add(Bidirectional(LSTM(100,return_sequences=True)))
model_b.add(Bidirectional(LSTM(160)))
model_b.add(Dense(2,activation='softmax'))
model_b.compile(optimizer='adam',
              #loss='binary_crossentropy',
              loss='CategoricalCrossentropy',
              metrics=['accuracy'])
print(model_b.summary())

In [ ]:
epoch=10
batch_size=32
history = model_b.fit(X_train, Y_train, batch_size=batch_size, epochs=epoch, validation_split=0.1, verbose=1 )

## BiLSTM  result

In [ ]:
accr = model_b.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
valid_predict= model_b.predict(X1)
print(valid_predict[:5])

In [ ]:
#Generate predictions for all samples
predictions1 = model_b.predict(X1)
print(predictions1)
predict_resultsb=predictions.argmax(axis=1)
print(predict_resultsb[100])

In [ ]:
from sklearn.metrics import classification_report
test['predict']= predict_resultsb
test['predict'] = np.where((test.predict == 0),'hate',test.predict)
test['predict'] = np.where((test.predict =='1'),'normal',test.predict)
test['predict'] = np.where((test.predict =='2' ),'offensive',test.predict)
test['predict'] = np.where((test.predict =='3'),'unsure',test.predict)
labels = ['hate','normal','offensive','unsure']
print(classification_report(test['label'].tolist(),test['predict'].tolist(),labels=labels))

In [ ]:
#X = "@USER ለህዝባችሁ የማትራሩ ፅንፈኞች ከስካራችሁ ስትነቁ ወየው ያኔ ግፋችሁ ከትግራይ ህዝብ አልፎ ሌላውንም እያዳረሰ ነው ይብላኝ ለነገው ትውልድ ተጋሩ በናንተ ስራ ጠላት ብዙ ሆኖ አንገቱን ደፍቴ ይኖራል"
#model = model_b
#yhat = model_b.predict(X)
predb = model_b.predict(X1)
predb

In [ ]:
predb = model_b.predict(X1)
predb= np.argmax(predb, axis = 1)
test_labels = np.argmax(Y1,axis = 1)
cm=confusion_matrix(test_labels,predb)
plot_confusion_matrix(cm,figsize=(8,8), class_names=labels)

# CNN 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import SpatialDropout1D
from keras.callbacks import EarlyStopping
from tensorflow.keras import layers
import tensorflow as tf
max_features =50000
embedding_dim = 100
sequence_length = 100

model_c = tf.keras.Sequential()
model_c.add(layers.Embedding(max_features + 1, embedding_dim, input_length=X.shape[1]))
model_c.add(layers.Conv1D(128, 10, activation='tanh'))
model_c.add(layers.GlobalMaxPooling1D())
model_c.add(layers.Dense(4, activation='tanh'))
model_c.add(layers.Dense(4, activation='sigmoid'))
model_c.compile(optimizer='adam',
              #loss='binary_crossentropy',
              loss='CategoricalCrossentropy',
              metrics=['accuracy'])
print(model_c.summary())
history = model_c.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)#,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)]

## CNN  result

In [ ]:
accr = model_c.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
valid_predict= model_c.predict(X1)
print(valid_predict[:5])

In [ ]:
#Generate predictions for all samples
predictions_c = model_c.predict(X1)
print(predictions_c)
predict_results_c=predictions_c.argmax(axis=1)
print(predict_results_c[100])

In [ ]:
from sklearn.metrics import classification_report
test['predict']= predict_results_c
test['predict'] = np.where((test.predict == 0),'hate',test.predict)
test['predict'] = np.where((test.predict =='1'),'normal',test.predict)
test['predict'] = np.where((test.predict =='2' ),'offensive',test.predict)
test['predict'] = np.where((test.predict =='3'),'unsure',test.predict)
labels = ['hate','normal','offensive','unsure']
print(classification_report(test['label'].tolist(),test['predict'].tolist(),labels=labels))

In [ ]:
pred_c1 = model_c.predict(X1)
pred_c1= np.argmax(pred_c1, axis = 1)
test_labels = np.argmax(Y1,axis = 1)
cm=confusion_matrix(test_labels,pred_c1)
plot_confusion_matrix(cm,figsize=(8,8), class_names=labels)

## Cnn loss and accuracy graph

In [ ]:
import matplotlib.pyplot as plt

plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test) # get our predictions
acc = accuracy_score(y_test, y_pred) 
print('Overall accuracy of RNN: {:.3f}'.format(acc))